<a href="https://colab.research.google.com/github/nickleus27/CST383_SportsData/blob/main/SportsDataNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BitByBit Team: Sports Data Notebook
<ul>
<li>Nick Anderson</li>
<li>Oscar Ochoa</li>
<li>Juan Duarte</li>
</ul>

<hr>

# Add Installs:

In [4]:
! git clone https://github.com/nickleus27/CST383_SportsData.git

Cloning into 'CST383_SportsData'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 23 (delta 4), reused 16 (delta 1), pack-reused 0
Receiving objects: 100% (23/23), 492.92 KiB | 4.18 MiB/s, done.
Resolving deltas: 100% (4/4), done.


# Add Imports:



In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
# add needed imports here

# Read Data:

In [7]:
df = pd.read_csv("CST383_SportsData/archive/spreadspoke_scores.csv")
# add more data here
df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13795,1/20/2024,2023,Division,True,San Francisco 49ers,24.0,21.0,Green Bay Packers,SF,-10.0,50.5,Levi's Stadium,False,59.0,8.0,77.0,NaN
13796,1/21/2024,2023,Division,True,Buffalo Bills,24.0,27.0,Kansas City Chiefs,BUF,-2.5,45.5,Highmark Stadium,False,25.0,11.0,67.0,NaN
13797,1/21/2024,2023,Division,True,Detroit Lions,31.0,23.0,Tampa Bay Buccaneers,DET,-6.0,49,Ford Field,False,72.0,0.0,NaN,indoor
13798,1/28/2024,2023,Conference,True,Baltimore Ravens,NaN,NaN,Kansas City Chiefs,NaN,NaN,NaN,M&T Bank Stadium,False,NaN,NaN,NaN,NaN
